# XRT Blop Demo

For ophyd beamline setup see: 
- https://github.com/NSLS-II/blop/blob/main/src/blop/sim/xrt_beamline.py
- https://github.com/NSLS-II/blop/blob/main/src/blop/sim/xrt_kb_model.py

The picture below displays beam from geometric source propagating through a pair of toroidal mirrors focusing the beam on screen. Simulation of a KB setup.

![xrt_blop_layout_w.jpg](../_static/xrt_blop_layout_w.jpg)

In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
from bluesky.callbacks import best_effort
from bluesky.callbacks.tiled_writer import TiledWriter
from bluesky.run_engine import RunEngine
from ophyd.utils import make_dir_tree
from tiled.client import from_uri
from tiled.server import SimpleTiledServer

# Initialize Tiled server and client
tiled_server = SimpleTiledServer(directory="/tmp/blop/sim/data/")
tiled_client = from_uri(tiled_server.uri)
tiled_writer = TiledWriter(tiled_client)
RE = RunEngine({})


# Set up RE enviornment
def setup_re_env(db_type="default", root_dir="/default/path"):
    bec = best_effort.BestEffortCallback()
    RE.subscribe(bec)
    RE.subscribe(tiled_writer)
    _ = make_dir_tree(datetime.datetime.now().year, base_path=root_dir)
    return {"RE": RE, "tiled": tiled_client, "bec": bec}


env = setup_re_env(db_type="temp", root_dir="/tmp/blop/sim/data")
globals().update(env)

bec.disable_plots()

Tiled version 0.1.dev2754+g895a852


2025-06-03 16:32:08.468 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/table/partition/ae98d7c1-253f-49b7-acea-a2efd3dd0731/primary/internal/events?partition=0 "HTTP/1.1 200 OK"
2025-06-03 16:32:08.573 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/array/block/ae98d7c1-253f-49b7-acea-a2efd3dd0731/primary/external/bl_det_image?block=9%2C0%2C0&expected_shape=1%2C300%2C400 "HTTP/1.1 200 OK"
2025-06-03 16:32:08.601 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/array/block/ae98d7c1-253f-49b7-acea-a2efd3dd0731/primary/external/bl_det_image?block=11%2C0%2C0&expected_shape=1%2C300%2C400 "HTTP/1.1 200 OK"
2025-06-03 16:32:08.644 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/array/block/ae98d7c1-253f-49b7-acea-a2efd3dd0731/primary/external/bl_det_image?block=4%2C0%2C0&expected_shape=1%2C300%2C400 "HTTP/1.1 200 OK"
2025-06-03 16:32:08.656 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/array/block/ae98d7c1-253f-49b7-acea-a2efd3dd0731/primary/external/bl_det_ima

In [2]:
import time

from matplotlib import pyplot as plt

from blop import DOF, Agent, Objective
from blop.digestion import beam_stats_digestion
from blop.sim.xrt_beamline import Beamline

/home/jmoylan1/miniconda3/envs/test_tiled_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
plt.ion()

h_opt = 0
dh = 5

R1, dR1 = 40000, 10000
R2, dR2 = 20000, 10000

In [4]:
beamline = Beamline(name="bl")
time.sleep(1)
dofs = [
    DOF(description="KBV R",
        device=beamline.kbv_dsv,
        search_domain=(R1 - dR1, R1 + dR1)),
    DOF(description="KBH R",
        device=beamline.kbh_dsh,
        search_domain=(R2 - dR2, R2 + dR2)),

]

In [5]:
objectives = [
    Objective(name="bl_det_sum",
              target="max",
              transform="log",
              trust_domain=(20, 1e12)),

    Objective(name="bl_det_wid_x",
              target="min",
              transform="log",
              # trust_domain=(0, 1e12),
              latent_groups=[("bl_kbh_dsh", "bl_kbv_dsv")]),
    Objective(name="bl_det_wid_y",
              target="min",
              transform="log",
              # trust_domain=(0, 1e12),
              latent_groups=[("bl_kbh_dsh", "bl_kbv_dsv")]),
]

In [6]:
# import json
# class JSONWriter:
#     """Writer for a JSON array"""

#     def __init__(self, filepath):
#         self.file = open(filepath, "w")
#         self.file.write("[\n")

#     def __call__(self, name, doc):
#         json.dump({"name": name, "doc": doc}, self.file)
#         if name == "stop":
#             self.file.write("\n]")
#             self.file.close()
#         else:
#             self.file.write(",\n")

# wr = JSONWriter('/tmp/blop/sim/test_tiled_writer1.json')
# RE.subscribe(wr)

In [7]:
agent = Agent(
    dofs=dofs,
    objectives=objectives,
    detectors=[beamline.det],
    digestion=beam_stats_digestion,
    digestion_kwargs={"image_key": "bl_det_image"},
    verbose=True,
    tiled=tiled,
    tolerate_acquisition_errors=False,
    enforce_all_objectives_valid=True,
    train_every=3,
)

In [12]:
RE(agent.learn("qr", n=16))

2025-06-03 16:33:02.040 INFO: Executing plan <generator object Agent.learn at 0x7d7d2bf44cc0>
2025-06-03 16:33:02.042 INFO: Change state on <bluesky.run_engine.RunEngine object at 0x7d7e2af5c4d0> from 'idle' -> 'running'
2025-06-03 16:33:02.042 INFO: running iteration 1 / 1


image_shape: (300, 400)
(1, 300, 400)


Transient Scan ID: 6     Time: 2025-06-03 16:33:02
Persistent Unique Scan ID: '216416f0-801e-4dca-94f3-8cef39f399fa'


2025-06-03 16:33:02.065 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata/ "HTTP/1.1 200 OK"


New stream: 'primary'
+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
|   seq_num |       time | bl_kbv_dsv | bl_kbh_dsh | bl_det_sum | bl_det_cen_x | bl_det_cen_y | bl_det_wid_x | bl_det_wid_y |
+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+


2025-06-03 16:33:02.102 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/search//216416f0-801e-4dca-94f3-8cef39f399fa?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-03 16:33:02.114 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa "HTTP/1.1 200 OK"
2025-06-03 16:33:02.125 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa/primary "HTTP/1.1 200 OK"
2025-06-03 16:33:02.137 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa/primary "HTTP/1.1 200 OK"
2025-06-03 16:33:02.149 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa/primary "HTTP/1.1 200 OK"
2025-06-03 16:33:02.161 INFO: HTTP Request: PATCH http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa/primary "HTTP/1.1 200 OK"
2025-06-03 16:33:02.170 INFO: HTTP Request: GET http://127.0.0.1:34059/ap

...

|        16 | 16:33:03.7 |  39986.755 |  11993.769 |      5.482 |      183.539 |      124.676 |      323.312 |      231.511 |


2025-06-03 16:33:03.729 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa/primary/internal "HTTP/1.1 200 OK"
2025-06-03 16:33:03.737 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/search/216416f0-801e-4dca-94f3-8cef39f399fa/primary/internal?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-03 16:33:03.745 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/216416f0-801e-4dca-94f3-8cef39f399fa/primary/internal/events "HTTP/1.1 200 OK"
2025-06-03 16:33:03.756 INFO: HTTP Request: PATCH http://127.0.0.1:34059/api/v1/table/partition/216416f0-801e-4dca-94f3-8cef39f399fa/primary/internal/events?partition=0 "HTTP/1.1 200 OK"


+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
generator list_scan ['216416f0'] (scan num: 6)





2025-06-03 16:33:03.871 INFO: HTTP Request: PATCH http://127.0.0.1:34059/api/v1/metadata//216416f0-801e-4dca-94f3-8cef39f399fa "HTTP/1.1 200 OK"
2025-06-03 16:33:03.880 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/216416f0-801e-4dca-94f3-8cef39f399fa "HTTP/1.1 200 OK"
2025-06-03 16:33:03.889 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/216416f0-801e-4dca-94f3-8cef39f399fa/primary/internal/events "HTTP/1.1 200 OK"
2025-06-03 16:33:03.911 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/216416f0-801e-4dca-94f3-8cef39f399fa "HTTP/1.1 200 OK"
2025-06-03 16:33:03.920 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/216416f0-801e-4dca-94f3-8cef39f399fa/primary/external/bl_det_image "HTTP/1.1 200 OK"
2025-06-03 16:33:04.303 INFO: Change state on <bluesky.run_engine.RunEngine object at 0x7d7e2af5c4d0> from 'running' -> 'idle'
2025-06-03 16:33:04.304 INFO: Cleaned up from plan <generator object Agent.learn at 0x7d7d2bf44cc0>


('216416f0-801e-4dca-94f3-8cef39f399fa',)

In [ ]:
# RE(agent.learn("qr", n=4, iterations=4))


2025-06-03 16:32:08.886 INFO: Executing plan <generator object Agent.learn at 0x7d7f1c3ab240>
2025-06-03 16:32:08.887 INFO: Change state on <bluesky.run_engine.RunEngine object at 0x7d7e2af5c4d0> from 'idle' -> 'running'
2025-06-03 16:32:08.887 INFO: running iteration 1 / 4


image_shape: (300, 400)
(1, 300, 400)


Transient Scan ID: 2     Time: 2025-06-03 16:32:08
Persistent Unique Scan ID: '8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340'


2025-06-03 16:32:08.903 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata/ "HTTP/1.1 200 OK"


New stream: 'primary'
+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
|   seq_num |       time | bl_kbv_dsv | bl_kbh_dsh | bl_det_sum | bl_det_cen_x | bl_det_cen_y | bl_det_wid_x | bl_det_wid_y |
+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+


2025-06-03 16:32:08.933 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/search//8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-03 16:32:08.943 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340 "HTTP/1.1 200 OK"
2025-06-03 16:32:08.952 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340/primary "HTTP/1.1 200 OK"
2025-06-03 16:32:08.962 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340/primary "HTTP/1.1 200 OK"
2025-06-03 16:32:08.971 INFO: HTTP Request: POST http://127.0.0.1:34059/api/v1/metadata//8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340/primary "HTTP/1.1 200 OK"
2025-06-03 16:32:08.981 INFO: HTTP Request: PATCH http://127.0.0.1:34059/api/v1/metadata//8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340/primary "HTTP/1.1 200 OK"
2025-06-03 16:32:08.989 INFO: HTTP Request: GET http://127.0.0.1:34059/ap

...

|         4 | 16:32:12.1 |  40194.772 |  28152.369 |      3.670 |      189.397 |      147.068 |      330.871 |      279.684 |


2025-06-03 16:32:12.205 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata//4b97fb35-4c9f-4e2d-8175-be751cd553ab/primary/internal "HTTP/1.1 200 OK"
2025-06-03 16:32:12.214 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/search/4b97fb35-4c9f-4e2d-8175-be751cd553ab/primary/internal?page%5Boffset%5D=0&fields=&sort= "HTTP/1.1 200 OK"
2025-06-03 16:32:12.223 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/4b97fb35-4c9f-4e2d-8175-be751cd553ab/primary/internal/events "HTTP/1.1 200 OK"
2025-06-03 16:32:12.234 INFO: HTTP Request: PATCH http://127.0.0.1:34059/api/v1/table/partition/4b97fb35-4c9f-4e2d-8175-be751cd553ab/primary/internal/events?partition=0 "HTTP/1.1 200 OK"


+-----------+------------+------------+------------+------------+--------------+--------------+--------------+--------------+
generator list_scan ['4b97fb35'] (scan num: 5)





2025-06-03 16:32:12.348 INFO: HTTP Request: PATCH http://127.0.0.1:34059/api/v1/metadata//4b97fb35-4c9f-4e2d-8175-be751cd553ab "HTTP/1.1 200 OK"
2025-06-03 16:32:12.358 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/4b97fb35-4c9f-4e2d-8175-be751cd553ab "HTTP/1.1 200 OK"
2025-06-03 16:32:12.368 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/4b97fb35-4c9f-4e2d-8175-be751cd553ab/primary/internal/events "HTTP/1.1 200 OK"
2025-06-03 16:32:12.390 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/4b97fb35-4c9f-4e2d-8175-be751cd553ab "HTTP/1.1 200 OK"
2025-06-03 16:32:12.399 INFO: HTTP Request: GET http://127.0.0.1:34059/api/v1/metadata/4b97fb35-4c9f-4e2d-8175-be751cd553ab/primary/external/bl_det_image "HTTP/1.1 200 OK"
2025-06-03 16:32:12.496 INFO: Change state on <bluesky.run_engine.RunEngine object at 0x7d7e2af5c4d0> from 'running' -> 'idle'
2025-06-03 16:32:12.497 INFO: Cleaned up from plan <generator object Agent.learn at 0x7d7f1c3ab240>


('8bcf2744-0c49-4e8c-b5b9-3ab8c1ec7340',
 'c4116607-21b1-4e6b-8ec8-37980feee8e7',
 'ad791c4d-edca-486f-b77b-119d89261593',
 '4b97fb35-4c9f-4e2d-8175-be751cd553ab')

In [13]:
agent.plot_objectives()

/home/jmoylan1/miniconda3/envs/test_tiled_new/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1631: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


AttributeError: 'NoneType' object has no attribute 'posterior'

Error in callback <function _draw_all_if_interactive at 0x7d7f08b0d940> (for post_execute), with arguments args (),kwargs {}:


ValueError: Invalid vmin or vmax

ValueError: Invalid vmin or vmax

<Figure size 3072x2304 with 15 Axes>